In [ ]:
import os
import torch
import numpy as np
from PIL import Image
from torchvision import transforms

def load_images_from_folder(folder, max_images=55):
    images = []
    for filename in os.listdir(folder)[:max_images]:
        img = Image.open(os.path.join(folder, filename))
        if img is not None:
            images.append(img)
    return images

transform_pipeline = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.Lambda(lambda x: x.convert('RGB')), # Преобразование в RGB
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Пути к папкам
Astana_path = "/content/drive/MyDrive/Astana"
Almata_path = "/content/drive/MyDrive/Almata"

# Загрузка изображений
Astana_images = load_images_from_folder(left_bank_path)
Almata_images = load_images_from_folder(Almata_path)

# Разделение на обучающую и тестовую выборки
train_images = Astana_images[:50] + Almata_images[:50]
test_images = Astana_images[50:] + Almata_images[50:]

# Преобразование изображений и создание тензоров
X_train = torch.stack([transform_pipeline(image) for image in train_images])
X_test = torch.stack([transform_pipeline(image) for image in test_images])

# Создание меток
y_train = torch.tensor([0]*50 + [1]*50)
y_test = torch.tensor([0]*5 + [1]*5)


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class Almaty_or_Astana(nn.Module):
    def __init__(self):
        super(Almaty_or_Astana, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1, stride = 1)
        self.bn1 = nn.BatchNorm2d(32)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1, stride = 1)
        self.bn2 = nn.BatchNorm2d(64)
        self.pool = nn.MaxPool2d(2, 2)
        self.dropout = nn.Dropout(0.25)
        self.fc1 = nn.Linear(64 * 32 * 32, 512) # Пример размера после пулинга и свертки
        self.bn3 = nn.BatchNorm1d(512)
        self.fc2 = nn.Linear(512, 2) # Два выхода: Астана и Алмата

    def forward(self, x):
        x = self.pool(F.relu(self.bn1(self.conv1(x))))
        x = self.pool(F.relu(self.bn2(self.conv2(x))))
        x = self.dropout(x)
        x = x.view(-1, 64 * 32 * 32)
        x = F.relu(self.bn3(self.fc1(x)))
        x = self.dropout(x)
        x = self.fc2(x)
        return x




In [ ]:
from torch import optim
model = Almaty_or_Astana()

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

print(model)

FakeOrRealCNN(
  (conv1): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (dropout): Dropout(p=0.25, inplace=False)
  (fc1): Linear(in_features=65536, out_features=512, bias=True)
  (bn3): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc2): Linear(in_features=512, out_features=2, bias=True)
)


In [ ]:
epochs = 50
for epoch in range(epochs):
    optimizer.zero_grad()

    outputs = model(X_train)
    loss = criterion(outputs, y_train)
    loss.backward()
    optimizer.step()

    print(f'Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}')

Epoch [1/50], Loss: 0.7701
Epoch [2/50], Loss: 0.4988
Epoch [3/50], Loss: 0.5859
Epoch [4/50], Loss: 0.2207
Epoch [5/50], Loss: 0.1549
Epoch [6/50], Loss: 0.1322
Epoch [7/50], Loss: 0.1203
Epoch [8/50], Loss: 0.0701
Epoch [9/50], Loss: 0.0578
Epoch [10/50], Loss: 0.0325
Epoch [11/50], Loss: 0.0240
Epoch [12/50], Loss: 0.0169
Epoch [13/50], Loss: 0.0126
Epoch [14/50], Loss: 0.0091
Epoch [15/50], Loss: 0.0057


KeyboardInterrupt: 

In [ ]:
from sklearn.metrics import f1_score, classification_report
import torch

# Переключение модель в режим оценки
model.eval()

# Отключение вычисления градиентов
with torch.no_grad():
    # Делаем предсказания для обучающей выборки
    outputs_train = model(X_train)
    _, predicted_train = torch.max(outputs_train, 1)

    # Делаем предсказания для тестовой выборки
    outputs_test = model(X_test)
    _, predicted_test = torch.max(outputs_test, 1)

# Переводим предсказания и истинные метки в формат numpy для использования с sklearn
predicted_train = predicted_train.numpy()
predicted_test = predicted_test.numpy()
y_train_np = y_train.numpy()
y_test_np = y_test.numpy()

# Вычисляем f1_score для обучающей выборки
f1_score_train = f1_score(y_train_np, predicted_train, average='binary')
print(f'Train F1 Score: {f1_score_train}')

# Вычисляем f1_score для тестовой выборки
f1_score_test = f1_score(y_test_np, predicted_test, average='binary')
print(f'Test F1 Score: {f1_score_test}')

# Выводим отчет классификации для обучающей выборки
print('\nClassification Report for Training Data:')
print(classification_report(y_train_np, predicted_train))

# Выводим отчет классификации для тестовой выборки
print('\nClassification Report for Test Data:')
print(classification_report(y_test_np, predicted_test))


Train F1 Score: 0.9433962264150945
Test F1 Score: 0.7272727272727272

Classification Report for Training Data:
              precision    recall  f1-score   support

           0       1.00      0.88      0.94        50
           1       0.89      1.00      0.94        50

    accuracy                           0.94       100
   macro avg       0.95      0.94      0.94       100
weighted avg       0.95      0.94      0.94       100


Classification Report for Test Data:
              precision    recall  f1-score   support

           0       0.75      0.60      0.67         5
           1       0.67      0.80      0.73         5

    accuracy                           0.70        10
   macro avg       0.71      0.70      0.70        10
weighted avg       0.71      0.70      0.70        10

